In [1]:
import os, sys

import pandas as pd
import math
import numpy as np
import random

from datetime import datetime

import itertools

from tqdm import tqdm, tqdm_notebook

<img src="nb_files/intro.jpg">
<style TYPE="text/css">
code.has-jax {font: inherit; font-size: 100%; background: inherit; border: inherit;}
</style>
<script type="text/x-mathjax-config">
MathJax.Hub.Config({
    tex2jax: {
        inlineMath: [['$','$'], ['\\(','\\)']],
        skipTags: ['script', 'noscript', 'style', 'textarea', 'pre'] // removed 'code' entry
    }
});
MathJax.Hub.Queue(function() {
    var all = MathJax.Hub.getAllJax(), i;
    for(i = 0; i < all.length; i += 1) {
        all[i].SourceElement().parentNode.className += ' has-jax';
    }
});
</script>
<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.4/MathJax.js?config=TeX-AMS_HTML-full"></script>


## Introduction

I landed at about 9 and missed pretty much the entire first Sunday of the regular season. The Dolphins had lost the 1 PM game by about 40 to the Ravens and the idea of them beating any other team in the league was unimaginable. As I got into my Uber home from the airport, I realized that meant picking against them was likely an effective but also popular strategy for a survivor league. I started counting the number of teams I could play against the Dolphins without repeats to see how long this strategy was viable. This quickly lead to the idea of an optimization challenge as it satisfies two main criteria:
1. It has a calculatable thing to maximize: evaluating a potential pick set can be done using data (though with questionable accuracy)
2. It has a constraint that makes it non-trivial: Being able to pick each team once is a simple rule that has just enough complexity to make things interesting

The optimization problem is defined by these two pieces and one extra parameter-: the time horizon. The length that the league goes for is a variable that adds a layer of complexity. If the expectation is for the league to last for 1 week, the best first pick will probably look different from the best first pick if the league were to last for 16 weeks.

For simplicity, we will make this an assumed parameter in our optimization. 

## Optimization

I am formalizing the problem here for anyone who cares to see it.

Let
$H = \text{Time Horizon,}$

$X_{tw} = \text{1 if team t is picked at week w, 0 otherwise}$

$P_{tw} = \text{Probability that team t wins at week w}$

$T = \text{the set of all teams}$

$T_{w} = \text{the set of teams playing at week w}$

### Objective Function

We would like to maximize the likelihood that our pickset survives the duration of the time.

$P(\text{pickset all wins})=P(\text{pick 1 wins, pick 2 wins,...})=P(\text{pick 1 wins})...P(\text{pick H wins})$

So in convoluted notation, our objective function is 

$\max {\prod\limits_{i=1}^{H} \sum\limits_{t \in T_i}  P_{ti}X_{ti}}$

### Constraints

The constraints are one team per week,

$ \sum\limits_{t \in T_i} X_{tw} = 1, \forall i \in [1,H]$

teams selected at most once,

$ \sum\limits_{i=1}^{H} X_{ti} \leq 1, \forall t \in T$


This looks like a pretty good linear programming question, which is not how I approached it.


## The data behind evaluation

FiveThirtyEight has maintained a set of ELO scores that characterize the strength of a team based on a historical data. ELO scores are designed to project probability of one team winning over another. This means that using their data and probability projections, I can score any pickset by calculating the joint probability as the product of the individual win probabilities of my picks. The optimization is clearly limited by the accuracy of the data, but as always, the data driven solution serves as a decision support tool rather than the decision itself.


In [2]:
f38_elo_snapshot = pd.read_csv('data/raw/nfl_elo_latest_2019-09-09.csv')

# Add a week variable
season_start =  datetime.strptime('2019-09-05', '%Y-%m-%d')
f38_elo_snapshot['days_into_season'] = [datetime.strptime(d, '%Y-%m-%d') - season_start for d in f38_elo_snapshot.date]
f38_elo_snapshot['week'] = [math.floor(dis.days / 7) + 1 for dis in f38_elo_snapshot.days_into_season]

f38_elo_snapshot = f38_elo_snapshot[f38_elo_snapshot.week <= 17].copy()

team1_elo = f38_elo_snapshot[['team1', 'elo1_post']]
team1_elo = team1_elo[team1_elo.elo1_post.notnull()].copy()
team1_elo.columns = ['team', 'elo']
team2_elo = f38_elo_snapshot[['team2', 'elo2_post']]
team2_elo = team2_elo[team2_elo.elo2_post.notnull()].copy()
team2_elo.columns = ['team', 'elo']
team_elo = team1_elo.append(team2_elo, ignore_index=True)
team_elo.sort_values('elo', ascending=False).reset_index(drop=True)

,team,elo
0,NE,1661.355272
1,KC,1622.148729
2,NO,1616.754464
3,LAR,1611.252716
4,LAC,1599.727360
5,BAL,1598.557710
6,PHI,1589.200034
7,SEA,1568.437337
8,DAL,1561.645312
9,MIN,1559.316056


## Is the problem trivial?

The problem is trivial if an intuitive, simple strategy can yield the optimal results. If the best strategy is something I can execute by looking at a list of numbers, then the problem does not need to be solved via optimization algorithms. 

The simplest greedy strategy is to pick the best matchup available each week. For a slightly better greedy baseline, I start with the most lop-sided matchup and eliminate the team and the week until every week has a pick. This will be what I call the greedy strategy for the remainder of the analysis.

I evaluate the greedy strategy by seeing where the chosen greedy best pickset falls in the universe of all valid picksets. At small time horizons, there are few enough pickset combinations to compare where the greedy solution ranks as the globally best pickset.



In [3]:
# Saving home team related variables so I can add that advantage
pick_candidates = f38_elo_snapshot[['week', 'team1', 'team2', 'neutral', 'elo_prob1','elo_prob2']]

pick_candidates_1 = pick_candidates[['week', 'team1', 'elo_prob1', 'neutral']].copy()
pick_candidates_1['is_team1'] = [False if neu == 1 else True for neu in pick_candidates_1.neutral]
pick_candidates_1.columns = ['week', 'team', 'probability', 'neutral','is_home']

pick_candidates_2 = pick_candidates[['week', 'team2', 'elo_prob2', 'neutral']].copy()
pick_candidates_2['is_home'] = False
pick_candidates_2.columns = ['week', 'team', 'probability', 'neutral','is_home']

pick_candidates = pick_candidates_1.append(pick_candidates_2).reset_index(drop=True)


### Greedy results
Below are the results for the best picksets according to the greedy strategy of picking the most lopsided matchups and eliminating that team and week from the selection set until each week has been picke.

In [4]:
def greedy_picker(n, force=None):
    # Assume the pickset needs to be the length n
    # and return the best set. If we already picked some
    # values, then let's store that in force and make sure the greedy
    # picker returns those values
    
    viable_candidates = pick_candidates[pick_candidates.week <= n]
    pickset = list()

    if force is not None:
        for elem in force:
            add_index = viable_candidates.index[(viable_candidates.week == elem['week']) & 
                                                        (viable_candidates.team == elem['team'])][0]
            
            pickset.append(viable_candidates.loc[add_index])
            selected_week = viable_candidates.loc[add_index]['week']
            selected_team = viable_candidates.loc[add_index]['team']
            
            print(selected_week)
            print(selected_team)
            viable_candidates = viable_candidates[(viable_candidates.team != selected_team) &
                                                 (viable_candidates.week != selected_week)]
            
    while len(pickset) < n:
        best_pick = viable_candidates.probability.idxmax()
        pickset.append(viable_candidates.loc[best_pick])
        selected_week = viable_candidates.loc[best_pick]['week']
        selected_team = viable_candidates.loc[best_pick]['team']
        viable_candidates = viable_candidates[(viable_candidates.team != selected_team) &
                                             (viable_candidates.week != selected_week)]
    results = pd.concat(pickset, axis=1,ignore_index=True).T
    results = results.sort_values(by='week')
    return results

def picks_to_string(picks):
    picks = ["{} - {}" .format(week, team) for week, team in zip(picks.week, picks.team)]
    return ', '.join(picks)
    
pick_results = list()
for i in range(1, 11):
    picks = greedy_picker(i)
    print(picks)
    picks_score = np.prod(picks.probability)
    print("Score: " + str(picks_score))
    picks = picks_to_string(picks)
    picks = str(i) + ': ' + picks
    pick_results.append(picks)

pick_results

  week team probability neutral is_home
0    1  PHI     0.76547       0    True
Score: 0.7654698543324963
  week team probability neutral is_home
1    1  PHI     0.76547       0    True
0    2  BAL    0.832054       0    True
Score: 0.6369120212695885
  week team probability neutral is_home
2    1  PHI     0.76547       0    True
1    2  BAL    0.832054       0    True
0    3   NE    0.881352       0    True
Score: 0.5613438281661964
  week team probability neutral is_home
3    1  PHI     0.76547       0    True
1    2  BAL    0.832054       0    True
0    3   NE    0.881352       0    True
2    4  LAR    0.822693       0    True
Score: 0.46181374931246544
  week team probability neutral is_home
4    1  SEA    0.761715       0    True
1    2  BAL    0.832054       0    True
0    3   NE    0.881352       0    True
3    4  LAR    0.822693       0    True
2    5  PHI    0.830608       0    True
Score: 0.3817042855851708
  week team probability neutral is_home
4    1  SEA    0.761715      

['1: 1 - PHI',
 '2: 1 - PHI, 2 - BAL',
 '3: 1 - PHI, 2 - BAL, 3 - NE',
 '4: 1 - PHI, 2 - BAL, 3 - NE, 4 - LAR',
 '5: 1 - SEA, 2 - BAL, 3 - NE, 4 - LAR, 5 - PHI',
 '6: 1 - SEA, 2 - BAL, 3 - NE, 4 - LAR, 5 - PHI, 6 - KC',
 '7: 1 - SEA, 2 - BAL, 3 - NE, 4 - LAR, 5 - PHI, 6 - KC, 7 - BUF',
 '8: 1 - SEA, 2 - BAL, 3 - NE, 4 - LAR, 5 - PHI, 6 - KC, 7 - BUF, 8 - NO',
 '9: 1 - CHI, 2 - BAL, 3 - NE, 4 - LAR, 5 - PHI, 6 - KC, 7 - BUF, 8 - NO, 9 - SEA',
 '10: 1 - CHI, 2 - BAL, 3 - NE, 4 - LAR, 5 - PHI, 6 - KC, 7 - BUF, 8 - NO, 9 - SEA, 10 - IND']

### Testing our greedy solution through brute force

For each time horizon that can be calculated in a reasonable amount of time (ie. until I think it takes too long), I will iterate through all combinations and see how they score on the objective function. Then I store the rank and better solutions found compared to the greedy solution.

In [5]:
# https://stackoverflow.com/questions/798854/all-combinations-of-a-list-of-lists

# List of sets of pickable teams
weekly_pickables = pick_candidates.groupby('week')['team'].apply(list)
# print([elem for elem in weekly_pickables[:3]])

# Dictionary of weekly probabilities for team: Week - Team : Prob
weekly_probabilities = {"{} - {}".format(week, team): prob for week, team, prob in zip(list(pick_candidates.week), 
                                                                                       list(pick_candidates.team), 
                                                                                       list(pick_candidates.probability))}
# print(weekly_probabilities)
def analyze_top_picks(n_weeks, force=None):
    greedy_picks = greedy_picker(n_weeks, force=force)
    greedy_score = np.prod(greedy_picks.probability)
    print("Greedy picks: " + picks_to_string(greedy_picks))
    print("Greedy score: " + str(greedy_score))
    
    all_sets = []
    scored_higher = 0
    total = 0
    
    weekly_candidates = weekly_pickables[:n_weeks].copy()
    if force is not None:
        # Replace each list with just the forced value
        for elem in force:
            print(elem['team'])
            print(list(elem['team']))
            weekly_candidates[elem['week']] = [elem['team']]
            
    for pickset in tqdm_notebook(itertools.product(*[elem for elem in weekly_candidates])):
        # Validate pickset
        
        if len(set(pickset)) == len(pickset):
        
            pickset_score = np.prod([weekly_probabilities['{} - {}'.format(i + 1, elem)] for i, elem in enumerate(pickset)])
    #         print(pickset_score)
            if pickset_score > greedy_score:
                scored_higher += 1
                all_sets.append({'solution': pickset, 'score': pickset_score})
                print("Found better")
            total += 1
    
    return({'better_sets' : all_sets,
           'n_better': scored_higher,
           'greedy_rank': scored_higher + 1,
           'total_sets': total})
        
# analyze_top_picks(3)

In [6]:
pickset_brute_force = [analyze_top_picks(i) for i in range(1, 7)]   # top 6

pickset_brute_force_results = pd.DataFrame(pickset_brute_force)
print(pickset_brute_force_results)

Greedy picks: 1 - PHI
Greedy score: 0.7654698543324963



Greedy picks: 1 - PHI, 2 - BAL
Greedy score: 0.6369120212695885



Greedy picks: 1 - PHI, 2 - BAL, 3 - NE
Greedy score: 0.5613438281661964



Greedy picks: 1 - PHI, 2 - BAL, 3 - NE, 4 - LAR
Greedy score: 0.46181374931246544



Greedy picks: 1 - SEA, 2 - BAL, 3 - NE, 4 - LAR, 5 - PHI
Greedy score: 0.3817042855851708


Found better

Greedy picks: 1 - SEA, 2 - BAL, 3 - NE, 4 - LAR, 5 - PHI, 6 - KC
Greedy score: 0.28070727461727835


Found better
Found better
Found better
Found better
Found better
Found better
Found better
Found better

                                         better_sets  n_better  greedy_rank  \
0                                                 []         0            1   
1                                                 []         0            1   
2                                                 []         0            1   
3                                                 []         0            1   
4  [{'solution': ('PHI', 'BAL', 'NE', 'LAR', 'NO'...         1            2   
5  [{'solution': ('CHI', 'BAL', 'DAL', 'LAR', 'NO...         8            9   

   total_sets  
0          32  
1         992  
2       29760  
3      809100  
4    21241920  
5   502147296  


### Greedy optimization analysis results

The greedy strategy yields the best results until the time horizon is 5 or more weeks. This suggests if we are searching for a solution for a surivivor league that is expected to go 5 or more weeks, we can't just go down the list of probabilities eliminating teams and weeks.


## Improving the greedy pickset

At the same time that the greedy algorithm stops working, the ability to brute force also conveniently disappears. The optimization space is BIG, think of the number of teams you can pick each week being around 32 and the number of permutations as around $_{32} P _H$. Or more simply, for the 6 weeks I ran the optimization, I scored 502,147,296 valid sets.

### Searching for a better pickset
There are many ways to efficiently explore a space, my first thought was to test a metaheuristic algorithm on the optimal greedy pickset. Metaheuristics are algorithms that will find better or equal solutions the longer you search with no guarantee of finding the global optimum. I decided to use a simulated annealer to iterate on the greedy best pickset. The short explanation of this algorithm is that on each iteration, you look for a slightly different solution and evaluate the new solution. If it is better, you accept the solution and that is your new current solution. If it is worse, you can accept to pivot to that solution with some probability to avoid getting stuck in a local optimum. As iterations pass, the probability of accepting a worse solution tends towards zero. The best solution is stored and updated throughout the process.

Using a simulated annealer, I would test minor changes to the best greedy pickset and see if I can beat it in a reasonable number of iterations. A new pickset is generated on each generation by swapping two teams in the set or picking up a completely new team to the set.

### Evaluating whether or not this works

My first step is to run it on a 6 week horizon to see if I can pick up a better solution than the greedy set. If I don't find any better solutions, then the implementation and parameters of the annealer are off.



In [7]:
def perturb_pickset(old_pickset, force=None):
    # This function returns a slightly different pickset
    # The goal is to swap two picks and validate that we didn't pick bye weeks
    
    p_swap = 0.2
    uniform = random.random()
    
    valid_swap = False
    
    ignore = []
    if force is not None:
        ignore = [elem['week'] - 1 for elem in force]
    
    index_candidates = list(range(len(old_pickset)))
    index_candidates = [elem for elem in index_candidates if not elem in ignore]
    
    if uniform < p_swap:
        while not valid_swap:
            indices_to_swap = random.sample(index_candidates, 2)
            valid_swap = old_pickset[indices_to_swap[0]] in weekly_pickables[indices_to_swap[1] + 1] 
            valid_swap = valid_swap and old_pickset[indices_to_swap[1]] in weekly_pickables[indices_to_swap[0] + 1]
            new_pickset = old_pickset.copy()
            new_pickset[indices_to_swap[0]], new_pickset[indices_to_swap[1]] = new_pickset[indices_to_swap[1]], new_pickset[indices_to_swap[0]]
        return new_pickset
    else:
        while not valid_swap:
            index_to_change = random.sample(index_candidates, 1)[0]
            candidates = weekly_pickables[index_to_change + 1].copy()
            new_elem = random.sample(candidates, 1)[0]
            valid_swap = new_elem not in old_pickset    # Avoid using already selected at any week including index
            new_pickset = old_pickset.copy()
            new_pickset[index_to_change] = new_elem
        return new_pickset

def temperature(t_0, i, cooling_rate):
    return t_0 * (cooling_rate  ** i)

def simulated_annealer(starting_pickset, max_iter=10000, force=None):
    
    current_score = np.prod([weekly_probabilities['{} - {}'.format(i + 1, elem)] for i, elem in enumerate(starting_pickset)])
    best_score = current_score
    
    
    current_solution = starting_pickset
    best_solution = current_solution
    
    annealer_results = [{'i': 0,
                        'score': current_score,
                        'best_score': best_score}]
    
    temp = -0.05 / np.log(0.20)    # I want a 20% chance of accepting a 5% decrease at start
    
    cooling_rate = (-0.05 / (temp * np.log(0.0001))) ** (1 / max_iter)   # At max iter I want a 0.01% chance   

    print("Temp: " + str(temp))
    
    print("Cooling rate: {}".format(cooling_rate))
    for iteration in tqdm_notebook(range(1, max_iter + 1)):
        
        new_solution = perturb_pickset(current_solution, force=force)
        new_score = np.prod([weekly_probabilities['{} - {}'.format(i + 1, elem)] for i, elem in enumerate(new_solution)])
        
        new_temp = temperature(temp, iteration, cooling_rate)
        
        accept_prob =  np.exp((new_score - current_score)/new_temp) 

        if random.random() < accept_prob:
            current_solution = new_solution
            current_score = new_score
            
        if new_score > best_score:
            print("Found better")
            best_score = new_score
            best_solution = current_solution
            print("New best score: " + str(best_score))
            print("New best solution: " + ', '.join(best_solution))
        
#         annealer_results.append({'i': iteration,
#                                  'score': current_score,
#                                  'best_score': best_score,
#                                 'acceptance': accept_prob})
        
    return {'starting_solution': starting_pickset,
           'best_solution': best_solution,
           'iteration_summary': annealer_results}


### Testing the simulated annealer

Let's test to see if we find the brute force top solutions for 6 week pickset in 1,000,000 iterations.

In [8]:
random.seed(100)
greedy_pickset = list(greedy_picker(6).team)
# perturb_pickset(greedy_pickset)
annealer_results = simulated_annealer(greedy_pickset, 1000000)


Temp: 0.031066746727980595
Cooling rate: 0.9999982555597104


Found better
New best score: 0.2909249896402786
New best solution: CHI, BAL, DAL, LAR, NO, NE
Found better
New best score: 0.2936182269111819
New best solution: PHI, BAL, DAL, LAR, NO, NE



### Results for evaluating the annealer

The above outputs show that we can find better solutions than the greedy solution. On future iterations, I should store the best value found to see if these are global optima.

## So how long should I expect the contest to go on for anyway?

At this point, it might be worth addressing what a reasonable value for the time horzion, $H$ is. 

Figuring out a good guess for how long we can go on is to estimate the longevity of likely opponent strategies. I just need to beat the 2nd place person to win. So what are reasonable strategies that the second best person could use?

1. <b>Greedy strategy</b>: If we really optimized things according to the estimates here, how long until it is unlikely that anyone is left? Looking at the greedy optimizer results, it looks like we have a 27.8% chance of making it until week 6.
2. <b>Dolphin streaming</b>: If you choose to pick against the dolphins and we assume they are as hopeless as we think, how long until you have to pick a repeat (ie. a non-risk free matchup)? Supposedly by their bye on week 5 or their repeat against the Bills in week 11, you would have to pick a team that could lose. I would assume week 5 you could find a reasonably good matchup, then this strategy could carry you to week 11.

So the scenarios we have to plan for are <b>6 Weeks and 11 Weeks.</b>

## So what am I supposed to do from here on out?

<img src='nb_files/chargers.jpeg'>


I picked the Chargers in week 1 because I felt like I wouldn't be confident in them again after their match against the Luck-less Colts. I needed to find what the next pick should be given that I already picked LAC week 1. Below I look at what the various strategies suggest.


### Greedy Selector for 6 Weeks
The greedy selector for 6 weeks suggest my next pick is <b>BAL</b>.

In [9]:
greedy_picker(6, force=[{'week': 1, 'team': 'LAC'}])

1
LAC


,week,team,probability,neutral,is_home
0,1,LAC,0.626264,0,True
2,2,BAL,0.832054,0,True
1,3,NE,0.881352,0,True
4,4,LAR,0.822693,0,True
3,5,PHI,0.830608,0,True
5,6,KC,0.735405,0,True


### Brute force over 6 weeks

Brute force over a 6 week horizon suggests that I should pick <b>BAL</b>.

In [10]:
analyze_top_picks(6, force=[{'week': 1, 'team': 'LAC'}])

1
LAC
Greedy picks: 1 - LAC, 2 - BAL, 3 - NE, 4 - LAR, 5 - PHI, 6 - KC
Greedy score: 0.23079109269671488
LAC
['L', 'A', 'C']


Found better
Found better



{'better_sets': [{'solution': ('LAC', 'BAL', 'DAL', 'LAR', 'NO', 'NE'),
   'score': 0.24022195187722442},
  {'solution': ('LAC', 'BAL', 'DAL', 'LAR', 'PHI', 'NE'),
   'score': 0.24119231677793931}],
 'n_better': 2,
 'greedy_rank': 3,
 'total_sets': 15553944}

### Simulated annealer over 11 week greedy pick

And yet again, my optimizers suggest <b>BAL</b> for week 2.

In [13]:
random.seed(100)
starting_pickset = list(greedy_picker(11, force=[{'week': 1, 'team': 'LAC'}]).team)
print(starting_pickset)
simulated_annealer(starting_pickset, 1000000, force=[{'week': 1, 'team': 'LAC'}])

1
LAC
['LAC', 'BAL', 'NE', 'LAR', 'PHI', 'KC', 'BUF', 'NO', 'SEA', 'IND', 'MIN']
Temp: 0.031066746727980595
Cooling rate: 0.9999982555597104


{'starting_solution': ['LAC',
  'BAL',
  'NE',
  'LAR',
  'PHI',
  'KC',
  'BUF',
  'NO',
  'SEA',
  'IND',
  'MIN'],
 'best_solution': ['LAC',
  'BAL',
  'NE',
  'LAR',
  'PHI',
  'KC',
  'BUF',
  'NO',
  'SEA',
  'IND',
  'MIN'],
 'iteration_summary': [{'i': 0,
   'score': 0.06195685260812797,
   'best_score': 0.06195685260812797}]}

## To be honest though, I don't feel that great about BAL in Week 2